In [1]:
import os
import glob
import numpy as np
import faiss # 圖片檢索的核心庫
import torch
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torchvision.models import resnet50

In [2]:
# --- 來自 model_pro-50.ipynb 的標準設定 ---
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 檢索使用的預處理 (與訓練集的驗證/測試集預處理一致)
VAL_TEST_TRANSFORMS = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
# ---------------------------------------------

# --- 檔案路徑設定 ---
MODEL_PATH = "pro-50.pth"
# 資料庫圖片路徑：假設 DeepFashion 結構為 deepfashion/images/
DATABASE_IMAGES_DIR = "deepfashion/images"
FAISS_DB_PATH = "faiss_deepfashion_retrieval.index"
FEATURE_DIMENSION = 2048 # ResNet50 倒數第二層的維度

def load_feature_extractor(model_path: str) -> nn.Module:
    """
    載入 pro-50.pth 權重到 ResNet50 模型，並移除最後的分類層。
    """
    print(f"1. 載入模型 ({MODEL_PATH})...")
    
    # 建立 ResNet50 架構，不使用 ImageNet 預訓練權重
    model = resnet50(weights=None)
    
    # 移除最後的全連接層 (Classification Layer)
    # 這樣模型輸出就是 Global Average Pooling 層之後的 2048 維特徵向量
    model.fc = nn.Identity() 

    # 載入 pro-50.pth 權重
    try:
        # 預設載入的是 state_dict (僅包含權重)
        state_dict = torch.load(model_path, map_location=DEVICE)
        
        # 由於我們修改了模型結構 (fc 變為 Identity)，state_dict 中的 fc 層權重會被忽略
        # 如果原始模型保存的是完整模型 (model.fc 存在)，需要確保 key 相符
        
        # 檢查 state_dict 的 key 是否包含 'fc.weight'。如果有，需要移除。
        # 由於我們不知道 pro-50.pth 是保存完整模型還是 state_dict，使用以下方法處理
        if 'fc.weight' in state_dict:
             state_dict.pop('fc.weight')
             state_dict.pop('fc.bias')
        
        model.load_state_dict(state_dict, strict=False) # strict=False 允許 keys 不完全匹配 (因為我們移除了 fc)
        
        model.to(DEVICE)
        model.eval() # 設為評估模式
        print(f"   ✅ 模型載入完成，輸出特徵維度: {FEATURE_DIMENSION}")
        return model
    except Exception as e:
        print(f"   ❌ 載入模型時發生錯誤。請確保 {MODEL_PATH} 檔案存在且為有效的 PyTorch 權重。")
        print(f"   錯誤訊息: {e}")
        return None

def get_image_vector(image_path: str, model: nn.Module) -> np.ndarray:
    """
    對單張圖片進行預處理並提取 2048 維特徵向量。
    """
    try:
        image = Image.open(image_path).convert("RGB")
        # 應用預處理
        input_tensor = VAL_TEST_TRANSFORMS(image)
        # 增加 batch 維度: (C, H, W) -> (1, C, H, W)
        input_batch = input_tensor.unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            # 執行模型推論，得到 2048 維特徵向量
            vector = model(input_batch).squeeze(0).cpu().numpy()
            
        return vector.astype('float32')
    except Exception as e:
        # print(f"處理圖片 {image_path} 失敗: {e}")
        return None

In [4]:
def build_faiss_index(image_dir: str, model: nn.Module) -> tuple:
    """
    遍歷資料庫圖片，提取向量，建立 FAISS 索引並儲存。
    """
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    if not image_paths:
        print(f"2. ❌ 在 {image_dir} 中找不到任何圖片 (.jpg)。請檢查路徑。")
        return None, None

    vectors = []
    file_names = []
    
    print(f"\n2. 正在轉換 {len(image_paths)} 張圖片為向量並建立索引...")
    # 使用 tqdm 顯示進度條
    for path in tqdm(image_paths, desc="提取特徵"):
        vector = get_image_vector(path, model)
        if vector is not None and vector.shape[0] == FEATURE_DIMENSION:
            vectors.append(vector)
            file_names.append(os.path.basename(path))

    if not vectors:
        print("   ❌ 未能成功提取任何圖片向量。")
        return None, None
        
    vectors_array = np.array(vectors)

    # 建立 FAISS 索引 (使用 IndexFlatL2 歐式距離)
    print(f"   建立 FAISS 索引，向量數: {vectors_array.shape[0]}, 維度: {FEATURE_DIMENSION}")
    index = faiss.IndexFlatL2(FEATURE_DIMENSION)
    index.add(vectors_array)
    
    # 儲存 FAISS 索引檔
    faiss.write_index(index, FAISS_DB_PATH)
    print(f"   ✅ FAISS 索引檔已儲存至 {FAISS_DB_PATH}")

    return index, file_names

# 執行建立資料庫
if os.path.exists(MODEL_PATH):
    feature_extractor_model = load_feature_extractor(MODEL_PATH)
    if feature_extractor_model:
        faiss_index, indexed_file_names = build_faiss_index(DATABASE_IMAGES_DIR, feature_extractor_model)
    else:
        faiss_index, indexed_file_names = None, None
else:
    print(f"❌ 錯誤: 找不到模型檔案 {MODEL_PATH}。請先確認檔案是否存在。")
    faiss_index, indexed_file_names = None, None

1. 載入模型 (pro-50.pth)...
   ✅ 模型載入完成，輸出特徵維度: 2048

2. 正在轉換 13752 張圖片為向量並建立索引...


提取特徵: 100%|██████████| 13752/13752 [21:06<00:00, 10.86it/s]


   建立 FAISS 索引，向量數: 13752, 維度: 2048
   ✅ FAISS 索引檔已儲存至 faiss_deepfashion_retrieval.index


In [5]:
def query_faiss_index(query_image_path: str, model: nn.Module, faiss_index: faiss.Index, indexed_names: list, k: int = 5) -> list:
    """
    使用查詢圖片查找 K 個最相似的資料庫圖片。
    """
    print(f"\n3. 執行向量查詢 (K={k})...")
    if faiss_index is None or indexed_names is None:
        print("   ❌ 向量資料庫尚未建立或載入。")
        return []

    # 1. 取得查詢圖片的向量
    query_vector = get_image_vector(query_image_path, model)
    if query_vector is None:
        print(f"   ❌ 無法提取查詢圖片 {query_image_path} 的向量。")
        return []
    
    # 將查詢向量轉換為 FAISS 期望的 (1, FEATURE_DIMENSION) 格式
    query_vector = query_vector.reshape(1, -1)
    
    # 2. 執行 FAISS 檢索
    # D: 距離 (Distance), I: 索引 (Index)
    D, I = faiss_index.search(query_vector, k)  
    
    # 3. 提取結果
    top_k_results = []
    for rank in range(k):
        index_in_db = I[0][rank]
        distance = D[0][rank]
        
        # 由於 FAISS 索引 0 是查詢圖片自己，如果距離極小，可以跳過 (在實際應用中可能不需要)
        # if distance < 1e-6: continue 
        
        file_name = indexed_names[index_in_db]
        
        top_k_results.append({
            "rank": rank + 1,
            "file_name": file_name,
            "distance_L2": float(distance) # L2 距離越小越相似
        })
        
    return top_k_results

# --- 範例查詢執行區塊 ---

# 請替換為您要查詢的圖片路徑
# 假設您從資料庫中選取一張圖片作為查詢範例
QUERY_IMAGE_EXAMPLE = os.path.join(DATABASE_IMAGES_DIR, "1000_031.jpg") 

if faiss_index and os.path.exists(QUERY_IMAGE_EXAMPLE):
    K_COUNT = 5 # 返回 5 個最相似的圖片

    results = query_faiss_index(
        query_image_path=QUERY_IMAGE_EXAMPLE,
        model=feature_extractor_model,
        faiss_index=faiss_index,
        indexed_names=indexed_file_names,
        k=K_COUNT
    )

    print(f"\n========== 查詢結果 (Top {K_COUNT}) ==========")
    print(f"查詢圖片: {os.path.basename(QUERY_IMAGE_EXAMPLE)}")
    for r in results:
        print(f"Rank {r['rank']}: {r['file_name']} (L2 距離: {r['distance_L2']:.4f})")
    print("==========================================")
elif faiss_index:
    print(f"\n❌ 錯誤: 範例查詢圖片路徑不存在: {QUERY_IMAGE_EXAMPLE}")


3. 執行向量查詢 (K=5)...

========== 查詢結果 (Top 5) ==========
查詢圖片: 1000_031.jpg
Rank 1: 1000_031.jpg (L2 距離: 0.0000)
Rank 2: 1000_032.jpg (L2 距離: 3262.7456)
Rank 3: 3059_032.jpg (L2 距離: 3388.4587)
Rank 4: 3059_031.jpg (L2 距離: 3527.3813)
Rank 5: 2244_021.jpg (L2 距離: 3644.5845)
